## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Romanian

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: `ro.deid.clinical`
- **Model Description**: This pipeline is trained with w2v_cc_300d Romanian embeddings and can be used to deidentify PHI information from medical texts in Romanian. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask, fake or obfuscate the following entities: AGE, CITY, COUNTRY, DATE, DOCTOR, EMAIL, FAX, HOSPITAL, IDNUM, LOCATION-OTHER, MEDICALRECORD, ORGANIZATION, PATIENT, PHONE, PROFESSION, STREET, ZIP, ACCOUNT, LICENSE, PLATE

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "ro-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'


    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data, indent=4, ensure_ascii=False) if content_type == 'application/json' else data


    if content_type == 'application/json':
        with open(input_file_name, "w", encoding='utf-8') as f:
            f.write(input_data)
    else:
        with open(input_file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(input_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=data)

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4, ensure_ascii=False)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    with open(output_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_output_dir}/{os.path.basename(output_file_name)}", Body=data)

    return response_data

### Initial Setup

In [7]:
docs = [
'''LABORATOR RADIOLOGIE, Strada Ababei, Sacueni, 354573 , TEL : 0265-210110 , E-MAIL: OFFICE@SMURD
Data... 
LABORATOR RADIOLOGIE, Strada Ababei, Sacueni, 354573 , TEL : 0265-210110 , E-MAIL: OFFICE@SMURD
Data setului de analize : 10 May 2022 18:45:00
Inregistrat de : PAUL DANIEL TUDOR 
Nume si prenume : MILASAN DANUT 
Varsta : 45 , Sex : Masculin
TEL : 456 45 789 2 
E-mail : damut__d@hotmail.com 
C.N.P : 1761218264378 
Cod pacient : 627480543615010
Licență : K0004567S, 
Înmatriculare : BD32904, 
Cont : SEWS324095192710408,
 
LA ORA : 10/12/2021 14:54_ 
VALIDAT DE :Dr. IOANA GHIBAN ; 
INVESTIGATII : CT TORACICÃ NATIVÃ''',

'''Medic : Dr. Agota EVELYN, 
C.N.P : 2450502264401, Data setului de analize: 25 May 2022 
Varsta : 77,... 
Medic : Dr. Agota EVELYN, 
C.N.P : 2450502264401, Data setului de analize: 25 May 2022 
Varsta : 77, Nume si Prenume : BUREAN MARIA 
Tel: +40(235)413773, 
E-mail : maria@gmail.com,
Licență : B004256985M, 
Înmatriculare : CD205113, 
Cont : FXHZ7170951927104999, 
Spitalul Pentru Ochi, Drumul de Deal ,Nr. 972 Vaslui, 737405

bolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s .
Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux .
Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min .
Opacifiere buna a patului vascular si cavitatilor cardiace .
Fara incidente sau accidente pe parcursul examinarii .

CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta .
Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator.'''

]


sample_text = """TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP 
Medic Laborator : BURIAN MONICA
PRO... 
TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP 
Medic Laborator : BURIAN MONICA
PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : 11 FEBRUARIE 2022 
Nume si prenume : RUS OLIMPIU DUMITRU , 49 ani, Sex : M, TEL : +40 67 5745, E-mail : olimpiu_d@gmail.com
CNP : 1730104060763, Licență : E000198985A, Înmatriculare : CD32156, Cont : LZWZ7170951927104999,
 
Solicitare : Angio CT coronarian , kg : 100 Dg . de trimitere Stare post-AVCI . Stare post-trombectomie . Proces inlocuitor de spatiu ventricular stang
Trimis de : Sectia Clinica Neurologie 1a SCJU Tirgu Mures
Medic : Dr.Melanie Dana
CONCLUZII : - Tromb ventricular stang de 20/40/17 mm , - Leziuni sechelar ischemice in segmentele apicale septal si inferior , cu aspect anevrismal apical VS - Leziune non-ischemica/MINOCA ?
48	mediocardiaca septala - Examenul AngioCT coronarian nu pune in evidenta ateromatoza sau stenoze coronariene semnificative."""

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**LABORATOR RADStrada AbabeidaSacueni, 354573i, 3545730265-21011065-210110 ,OFFICE@SMURDICE@SMURD**"

    - **masked**: Default policy that masks entities with their type.

      -> 'LABORATOR RADIOLOGIE, `<STREET>`, `<CITY>`, `<ZIP>` , TEL : `<PHONE>` , E-MAIL: `<EMAIL>`'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'LABORATOR RADIOLOGIE, `Intrarea Diaconescu`, `Aiud`, `302784` , TEL : `0263 144 119` , E-MAIL: `jeneltudor@email.ro`'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'LABORATOR RADIOLOGIE, `****`, `****`, `****` , TEL : `****` , E-MAIL: `****`'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'LABORATOR RADIOLOGIE, [`***********`], [`*****`], [`****`] , TEL : [`*********`] , E-MAIL: [`**********`]'

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}
```

### JSON


1. **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

2. **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

#### Example 1: masked (default-policy)

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP Medic Laborator : <DOCTOR> PRO. .. TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP Medic Laborator : <DOCTOR> PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : <DATE> Nume si prenume : <PATIENT> , <AGE> ani, Sex : M, TEL : <PHONE>, E-mail : <EMAIL> CNP : <LICENSE>, Licență : <LICENSE>, Înmatriculare : <PLATE>, Cont : <ACCOUNT>,\n \nSolicitare : Angio CT coronarian , kg : 100 Dg . de trimitere Stare post-AVCI . Stare post-trombectomie . Proces inlocuitor de spatiu ventricular stang Trimis de : Sectia Clinica Neurologie 1a <HOSPITAL> Medic : <DOCTOR> CONCLUZII : - Tromb ventricular stang de 20/40/17 mm , - Leziuni sechelar ischemice in segmentele apicale septal si inferior , cu aspect anevrismal apical VS - Leziune non-ischemica/MINOCA ? 48\tmediocardiaca septala - Examenul AngioCT coronarian nu pune in evidenta ateromatoza sau stenoze coronariene semnificative."


#### Example 2: obfuscated

In [9]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP Medic Laborator : Doina Gheorghiu PRO. .. TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP Medic Laborator : Doina Gheorghiu PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : 11 FEBRUARIE 2022 Nume si prenume : Dinut Liliana Daniela , 83 ani, Sex : M, TEL : +16 10 9604, E-mail : lstancu@zzup.ro CNP : 5409811914782, Licență : N562130865H, Înmatriculare : QI69629, Cont : SEAE8282288015490352,\n \nSolicitare : Angio CT coronarian , kg : 100 Dg . de trimitere Stare post-AVCI . Stare post-trombectomie . Proces inlocuitor de spatiu ventricular stang Trimis de : Sectia Clinica Neurologie 1a Institutul Clinic de Urologie si Transplant Renal Medic : Agata Voinea CONCLUZII : - Tromb ventricular stang de 20/40/17 mm , - Leziuni sechelar ischemice in segmentele apicale septal si inferior , cu aspect anevrismal apical VS - Leziune non-ischemica/MINOCA ? 48\tmediocardiaca septala - Examenul AngioCT coronarian nu pune in evidenta ateromatoza sau stenoze coronariene semnificative."


#### Example 3: masked (default-policy)

In [10]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data. .. LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data setului de analize : <DATE> 18:45:00 Inregistrat de : <DOCTOR> <DOCTOR> Nume si prenume : <PATIENT> Varsta : <AGE> , Sex : Masculin TEL : <PHONE> E-mail : <EMAIL> C.N.P : <IDNUM> Cod pacient : <IDNUM> Licență : <LICENSE>, Înmatriculare : <PLATE>, \nCont : <ACCOUNT>,\n \nLA ORA : <DATE> 14:54_ VALIDAT DE :Dr. <DOCTOR> ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE><PATIENT> Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE>, Nume si Prenume : <PATIENT> Tel: <PHONE>, \nE-mail : <EMAIL>,\nLicență : <LICENSE>, \nÎnmatriculare : <PLATE>, \nCont : <ACCOUNT>, \n<HOSPITAL> ,<STREET> <CITY>, <ZIP>\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


#### Example 4: obfuscated

In [11]:
input_json_data = {"text": docs, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"LABORATOR RADIOLOGIE, Drumul Tudor, Aiud, 161096 , TEL : 0454-098119 , E-MAIL: jeneltudor@email.ro Data. .. LABORATOR RADIOLOGIE, Drumul Tudor, Aiud, 147829 , TEL : 5621-308657 , E-MAIL: jeneltudor@email.ro Data setului de analize : 10 May 2022 18:45:00 Inregistrat de : EFTIMIE, Sinică Doina Gheorghiu Nume si prenume : Dobrin Figan Varsta : 94 , Sex : Masculin TEL : 846 96 295 2 E-mail : hcristea@141.ro C.N.P : 8413244010272 Cod pacient : 536644034742595 Licență : G3875643P, Înmatriculare : IR51884, \nCont : OATD2344587666964477,\n \nLA ORA : 10/12/2021 14:54_ VALIDAT DE :Dr. Iurie Caius Stoica ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"Medic : Dr. R.T., \nC.N.P : 1660630160109, Data setului de analize: 25 May 2022 Varsta : 91Dinut Liliana Daniela Medic : Dr. R.T., \nC.N.P : 3235573220254, Data setului de analize: 25 May 2022 Varsta : 91, Nume si Prenume : Dragan Mihai Tel: +27(062)376283, \nE-mail : jeneltudor@email.ro,\nLicență : T517616073X, \nÎnmatriculare : TG626948, \nCont : KHHO5029180812813651, \nInstitutul Clinic de Urologie si Transplant Renal ,Intrarea Diaconescu Curcani, 546270\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


### JSON Lines

1. **Input format**: Provide input in JSON Lines format, where each line is a JSON object representing a text document along with any optional parameters.

```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [12]:
import json

def create_jsonl(records, masking_policy):
    json_records = []

    for text in records:
        record = {
            "text": text,
            "masking_policy": masking_policy
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

#### Example 1: masked (default-policy)

In [13]:
input_jsonl_data = create_jsonl(docs, masking_policy="masked")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data. .. LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data setului de analize : <DATE> 18:45:00 Inregistrat de : <DOCTOR> <DOCTOR> Nume si prenume : <PATIENT> Varsta : <AGE> , Sex : Masculin TEL : <PHONE> E-mail : <EMAIL> C.N.P : <IDNUM> Cod pacient : <IDNUM> Licență : <LICENSE>, Înmatriculare : <PLATE>, \nCont : <ACCOUNT>,\n \nLA ORA : <DATE> 14:54_ VALIDAT DE :Dr. <DOCTOR> ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"}
{"predictions": "Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE><PATIENT> Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE>, Nume si Prenume : <PATIENT> Tel: <PHONE>, \nE-mail : <EMAIL>,\nLicență : <LICENSE>, \nÎnmatriculare : <PLATE>, \nCont : <ACCOUNT>, \n<HOSPITAL> ,<STREET> <CITY>, <ZIP>\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua 

#### Example 2: obfuscated

In [14]:
input_jsonl_data = create_jsonl(docs, masking_policy="obfuscated")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "LABORATOR RADIOLOGIE, Drumul Tudor, Aiud, 161096 , TEL : 0454-098119 , E-MAIL: jeneltudor@email.ro Data. .. LABORATOR RADIOLOGIE, Drumul Tudor, Aiud, 147829 , TEL : 5621-308657 , E-MAIL: jeneltudor@email.ro Data setului de analize : 10 May 2022 18:45:00 Inregistrat de : EFTIMIE, Sinică Doina Gheorghiu Nume si prenume : Dobrin Figan Varsta : 94 , Sex : Masculin TEL : 846 96 295 2 E-mail : hcristea@141.ro C.N.P : 8413244010272 Cod pacient : 536644034742595 Licență : G3875643P, Înmatriculare : IR51884, \nCont : OATD2344587666964477,\n \nLA ORA : 10/12/2021 14:54_ VALIDAT DE :Dr. Iurie Caius Stoica ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"}
{"predictions": "Medic : Dr. R.T., \nC.N.P : 1610960454098, Data setului de analize: 25 May 2022 Varsta : 91Dinut Liliana Daniela Medic : Dr. R.T., \nC.N.P : 1191478295621, Data setului de analize: 25 May 2022 Varsta : 91, Nume si Prenume : Dragan Mihai Tel: +30(865)784696, \nE-mail : jeneltudor@email.ro,\nLicență : E952841324M, \nÎnmatric

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [16]:
import json
import os


input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)


validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"


def write_and_upload_to_s3(input_data, file_name):

    file_format = os.path.splitext(file_name)[1].lower()
    s3_key = f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}"

    if file_format == ".json":
        input_data_json = json.dumps(input_data, indent=4, ensure_ascii=False)

        with open(file_name, "w", encoding='utf-8') as f:
            f.write(input_data_json)
    elif file_format == ".jsonl":
        with open(file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_key, Body=data)

    return input_data


input_json_data = {
    "input1.json": {"text": docs, "masking_policy": "masked"},
    "input2.json": {"text": docs, "masking_policy": "obfuscated"},
    "input3.json": {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    "input4.json": {"text": docs, "masking_policy": "masked_with_chars"}
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}


for file_name, data in input_json_data.items():
    write_and_upload_to_s3(data, f"{json_input_dir}/{file_name}")

for file_name, data in input_jsonl_data.items():
    write_and_upload_to_s3(data, f"{jsonl_input_dir}/{file_name}")



### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [18]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    
    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

In [19]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_json_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked


,predictions
0,"LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data. .. LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data setului de analize : <DATE> 18:45:00 Inregistrat de : <DOCTOR> <DOCTOR> Nume si prenume : <PATIENT> Varsta : <AGE> , Sex : Masculin TEL : <PHONE> E-mail : <EMAIL> C.N.P : <IDNUM> Cod pacient : <IDNUM> Licență : <LICENSE>, Înmatriculare : <PLATE>, \nCont : <ACCOUNT>,\n \nLA ORA : <DATE> 14:54_ VALIDAT DE :Dr. <DOCTOR> ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE><PATIENT> Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE>, Nume si Prenume : <PATIENT> Tel: <PHONE>, \nE-mail : <EMAIL>,\nLicență : <LICENSE>, \nÎnmatriculare : <PLATE>, \nCont : <ACCOUNT>, \n<HOSPITAL> ,<STREET> <CITY>, <ZIP>\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."




Masking policy: obfuscated


,predictions
0,"LABORATOR RADIOLOGIE, Drumul Tudor, Aiud, 161096 , TEL : 0454-098119 , E-MAIL: jeneltudor@email.ro Data. .. LABORATOR RADIOLOGIE, Drumul Tudor, Aiud, 147829 , TEL : 5621-308657 , E-MAIL: jeneltudor@email.ro Data setului de analize : 10 May 2022 18:45:00 Inregistrat de : EFTIMIE, Sinică Doina Gheorghiu Nume si prenume : Dobrin Figan Varsta : 94 , Sex : Masculin TEL : 846 96 295 2 E-mail : hcristea@141.ro C.N.P : 8413244010272 Cod pacient : 536644034742595 Licență : G3875643P, Înmatriculare : IR51884, \nCont : OATD2344587666964477,\n \nLA ORA : 10/12/2021 14:54_ VALIDAT DE :Dr. Iurie Caius Stoica ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"Medic : Dr. R.T., \nC.N.P : 1610960454098, Data setului de analize: 25 May 2022 Varsta : 91Dinut Liliana Daniela Medic : Dr. R.T., \nC.N.P : 1191478295621, Data setului de analize: 25 May 2022 Varsta : 91, Nume si Prenume : Dragan Mihai Tel: +30(865)784696, \nE-mail : jeneltudor@email.ro,\nLicență : E952841324M, \nÎnmatriculare : WN027253, \nCont : KHHO5029180812813651, \nInstitutul Clinic de Urologie si Transplant Renal ,Intrarea Diaconescu Curcani, 664403\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."




Masking policy: masked_fixed_length_chars


,predictions
0,"LABORATOR RADIOLOGIE, ****, ****, **** , TEL : **** , E-MAIL: **** Data. .. LABORATOR RADIOLOGIE, ****, ****, **** , TEL : **** , E-MAIL: **** Data setului de analize : **** 18:45:00 Inregistrat de : **** **** Nume si prenume : **** Varsta : **** , Sex : Masculin TEL : **** E-mail : **** C.N.P : **** Cod pacient : **** Licență : ****, Înmatriculare : ****, \nCont : ****,\n \nLA ORA : **** 14:54_ VALIDAT DE :Dr. **** ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"Medic : Dr. ****, \nC.N.P : ****, Data setului de analize: **** Varsta : ******** Medic : Dr. ****, \nC.N.P : ****, Data setului de analize: **** Varsta : ****, Nume si Prenume : **** Tel: ****, \nE-mail : ****,\nLicență : ****, \nÎnmatriculare : ****, \nCont : ****, \n**** ,**** ****, ****\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."




Masking policy: masked_with_chars


,predictions
0,"LABORATOR RADIOLOGIE, [***********], [*****], [****] , TEL : [*********] , E-MAIL: [**********] Data. .. LABORATOR RADIOLOGIE, [***********], [*****], [****] , TEL : [*********] , E-MAIL: [**********] Data setului de analize : [*********] 18:45:00 Inregistrat de : [*********] [***] Nume si prenume : [***********] Varsta : ** , Sex : Masculin TEL : [**********] E-mail : [******************] C.N.P : [***********] Cod pacient : [*************] Licență : [*******], Înmatriculare : [*****], \nCont : [*****************],\n \nLA ORA : [********] 14:54_ VALIDAT DE :Dr. [**********] ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"
1,"Medic : Dr. [**********], \nC.N.P : [***********], Data setului de analize: [*********] Varsta : **[**] Medic : Dr. [**********], \nC.N.P : [***********], Data setului de analize: [*********] Varsta : **, Nume si Prenume : [**********] Tel: [************], \nE-mail : [*************],\nLicență : [*********], \nÎnmatriculare : [******], \nCont : [******************], \n[**********************************] ,[*****] [****], [****]\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin cu 5 ml/s . Se continua cu 65 ml contrast si 30 ml flush salin , cu acelasi flux . Se efectueaza o examinare angio-CT coronariana cu achizitii spirale cu reconstructii retrospective la o frecventa cardiaca medie de 60/min . Opacifiere buna a patului vascular si cavitatilor cardiace . Fara incidente sau accidente pe parcursul examinarii . CONCLUZII : - Stare dupa stentare ramus intermedius si artera coronara dreapta . Aspect corespunzator al stenturilor , cu minima stenoza la nivelul ACD si cu flux distal corespunzator."


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [21]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [22]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_jsonlines_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked
{"predictions": "LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data. .. LABORATOR RADIOLOGIE, <STREET>, <CITY>, <ZIP> , TEL : <PHONE> , E-MAIL: <EMAIL> Data setului de analize : <DATE> 18:45:00 Inregistrat de : <DOCTOR> <DOCTOR> Nume si prenume : <PATIENT> Varsta : <AGE> , Sex : Masculin TEL : <PHONE> E-mail : <EMAIL> C.N.P : <IDNUM> Cod pacient : <IDNUM> Licență : <LICENSE>, Înmatriculare : <PLATE>, \nCont : <ACCOUNT>,\n \nLA ORA : <DATE> 14:54_ VALIDAT DE :Dr. <DOCTOR> ; \nINVESTIGATII : CT TORACICÃ NATIVÃ"}
{"predictions": "Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE><PATIENT> Medic : Dr. <DOCTOR>, \nC.N.P : <IDNUM>, Data setului de analize: <DATE> Varsta : <AGE>, Nume si Prenume : <PATIENT> Tel: <PHONE>, \nE-mail : <EMAIL>,\nLicență : <LICENSE>, \nÎnmatriculare : <PLATE>, \nCont : <ACCOUNT>, \n<HOSPITAL> ,<STREET> <CITY>, <ZIP>\n\nbolus 10 ml Iomeron 350 urmate de 30 ml flush salin c

In [23]:
model.delete_model()

INFO:sagemaker:Deleting model with name: ro-deid-clinical-2024-05-21-07-14-05-940


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

